In [20]:
import kfp
import kfp.components as comp
import kfp.dsl as dsl

In [21]:
def treina(feat:str, label:str,file:str) -> (float,float) :
    import pandas
    from sklearn.linear_model import LinearRegression
    df = pandas.read_csv(file)
    reglin = LinearRegression()
    reglin.fit(df[[feat]], df[label])
    return (reglin.coef_[0], reglin.intercept_)

In [22]:
comp_treina = comp.create_component_from_func(treina,output_component_file='treina_component.yaml',base_image='tensorflow/tensorflow:1.11.0-py3')

In [23]:
def valida (rl: (float,float),feat:str, label:str, file:str) -> float :
    import pandas
    from sklearn.linear_model import LinearRegression
    df = pandas.read_csv(file)
    reglin = LinearRegression()
    reglin.fit(df[[feat]], df[label])
    r2 = reglin.score(df[[feat]], df[label])
    return r2

In [24]:
comp_valida = comp.create_component_from_func(valida,output_component_file='valida_component.yaml',base_image='tensorflow/tensorflow:1.11.0-py3')

In [25]:
def previsao(r2:float, feat:str, label:str, valor:float, file:str) -> float:
    import pandas
    from sklearn.linear_model import LinearRegression
    df = pandas.read_csv(file)
    reglin = LinearRegression()
    reglin.fit(df[[feat]], df[label])
    future = pandas.DataFrame({feat:[valor]})
    return reglin.predict(future)[0]
    

In [26]:
comp_previsao = comp.create_component_from_func(previsao,output_component_file='previsao_component.yaml',base_image='tensorflow/tensorflow:1.11.0-py3')

In [27]:
@dsl.pipeline(name='Pipeline Data',description='Dados Bolsa X Dolar')
def pipe(feat, label, valor, file):
    task_treina   = comp_treina(feat,label,file)
    task_valida   = comp_valida(task_treina.output,feat,label,file)
    task_previsao = comp_previsao(task_valida.output,feat,label,valor,file)

In [29]:
client = kfp.Client(host='http://localhost:8080/pipeline')
FILE = 'https://raw.githubusercontent.com/ambientelivre/samples_kubeflow/main/datasets/bolsa.csv'
args = {'feat':'bolsa','label':'usd','valor':'110','file':FILE}
##print(client.list_experiments())
client.create_run_from_pipeline_func(pipe,arguments=args)

RunPipelineResult(run_id=52e07586-a114-4d29-a393-b7356c877ae5)